In [1]:
import argparse
import os
import math
import json
import pandas as pd
import random as rn
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader,SequentialSampler # RandomSampler

from sklearn. metrics import recall_score,precision_score, f1_score,plot_precision_recall_curve,roc_auc_score
import matplotlib.pyplot as plt

from data import *
from untils import *
from model import *
from loss import *

In [3]:
import torchvision.utils as vutils
import torchvision.models as models
from torchvision import datasets
from torch.utils.tensorboard import SummaryWriter

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=50, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=128, help="size of the batches")
parser.add_argument("--lrD", type=float, default=0.0005, help="discriminator adam: learning rate")
parser.add_argument("--lrG", type=float, default=0.0005, help="generator adam: learning rate")
parser.add_argument("--lrC", type=float, default=0.001, help="classifier adam: learning rate")
parser.add_argument("--wdG", type=float, default=5e-4, help="generator adam: weight_decay")
parser.add_argument("--wdD", type=float, default=5e-4, help="discriminator adam: weight_decay")
parser.add_argument("--wdC", type=float, default=5e-4, help=" classifier adam: weight_decay")
parser.add_argument("--p_hint", type=float, default=0.5, help="model: probability of hint")
parser.add_argument("--k", type=int, default=5, help="model:k for G&C running per iteration")
parser.add_argument("--alpha", type=int, default=5, help="model: alpha parameter reconstructed loss")
parser.add_argument("--beta", type=int, default=1, help="model: beta parameter classifier loss")
parser.add_argument("--n_class", type=int, default=1, help="label class")
parser.add_argument('--H_Dim1', type=int, default=32, help='hiden layer dimension1')
parser.add_argument('--H_Dim2', type=int, default=32, help='hiden layer dimension2')
parser.add_argument('--H_Dim3', type=int, default=64, help='hiden layer dimension3')
parser.add_argument("--dropoutD", type=float, default=0.1, help="dropout: NetD dropout")
parser.add_argument("--dropoutG", type=float, default=0.1, help="dropout: NetD dropout")
parser.add_argument("--dropoutC", type=float, default=0.1, help="dropout: NetD dropout")
parser.add_argument("--file_x", type=str, default="6hr", help="file: file feature name")
parser.add_argument("--file_m", type=str, default="6hr20", help="file: file mask name")
parser.add_argument("--file_y", type=str, default="6hr", help="file: file label name")
parser.add_argument("--thold", type=float, default=0.5, help="logits threshold")
opt, unknown = parser.parse_known_args()

In [9]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)
torch.cuda.manual_seed_all(321)
torch.backends.cudnn.deterministic=True

In [10]:
#cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cuda = True if torch.cuda.is_available() else False

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor




# data loading


In [11]:
trainX, trainy, devX, devy, testX, testy, trainX_M,testX_M,devX_M=data_load(opt.file_x,opt.file_y, opt.file_m,mask=True)

# network loading

In [16]:
no,dim=trainX.shape
netG = NetG(Dim=dim,H_Dim=opt.H_Dim1,dropout=opt.dropoutG)
netD = NetD(Dim=dim,H_Dim=opt.H_Dim2,n_class=opt.n_class,dropout=opt.dropoutD)
netC = NetC(Dim=dim,H_Dim=opt.H_Dim3,n_class=opt.n_class, dropout=opt.dropoutC)

In [19]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if use_cuda:
    netD = netD.cuda()
    netG = netG.cuda()
    netC = netC.cuda()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor

In [20]:

# convert all data to tensor type
trainX_tensor = torch.tensor(trainX).float()
trainy_tensor = torch.tensor(trainy).float()
trainM_tensor = torch.tensor(trainX_M).float()


devX_tensor = torch.tensor(devX).float()
devy_tensor = torch.tensor(devy).float()
devM_tensor = torch.tensor(devX_M).float()

testX_tensor = torch.tensor(testX).float()
testy_tensor = torch.tensor(testy).float()
testM_tensor = torch.tensor(testX_M).float()

In [22]:
optimD = torch.optim.Adam(netD.parameters(), lr=opt.lrD, weight_decay=opt.wdD) 
optimG = torch.optim.Adam(netG.parameters(), lr=opt.lrG,weight_decay=opt.wdG)
optimC = torch.optim.Adam(netC.parameters(), lr=opt.lrC,weight_decay=opt.wdC)

In [23]:
#data loading
train_dataset = TensorDataset(trainX_tensor, trainM_tensor, trainy_tensor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=opt.batch_size) 

dev_dataset = TensorDataset(devX_tensor,  devM_tensor, devy_tensor)
dev_sampler = SequentialSampler(dev_dataset)
dev_dataloader = DataLoader(dev_dataset, sampler=dev_sampler, batch_size=opt.batch_size)

# Training

In [24]:
writer = SummaryWriter()
auc_max=0
f1_max=0
accumulation_step=opt.k
global_step=0
for epoch_num in range(opt.n_epochs):
        
    netD.train()
    netG.train()
    netC.train()
    

    test_loss_mse=0
    train_loss_D=0
    predict_loss=0
    for step_num, batch_data in list(enumerate(train_dataloader))[:-1]:
        global_step+=1
        torch.cuda.empty_cache()
        X, M,y= tuple(t.to(device) for t in batch_data)
        B = sample_binary(X.shape[0], dim, 1-opt.p_hint)
        B = torch.tensor(B,device="cuda").float()
        H =M *B+0.5*(1-B)
        
        
        # -----------------
        #  Train G&C
        # -----------------

        optimG.zero_grad()
        optimC.zero_grad()
        
        
        class_loss_sum=0
        reconstruct_loss_sum=0
        adversarial_G_loss_sum=0
        for i in range(accumulation_step):
            Z= Variable(FloatTensor(np.random.uniform(0, 1, (X.shape[0], dim))))
        
            New_X = M * X + (1-M) * Z
            G_sample = netG(New_X, M)
            Hat_New_X = New_X*M + G_sample * (1-M)
            C_logits=netC(Hat_New_X)
            D_logit = netD(Hat_New_X, H,C_logits) 
        
            adversarial_G_loss,reconstruct_loss=generator_loss(New_X, M, G_sample, D_logit)
#             adversarial_G_loss =(-torch.mean((1 - M) * (torch.sigmoid(D_logit)+1e-8).log())/(1-M).mean())
#             reconstruct_loss=mse_loss(M*New_X, M*G_sample)/ torch.mean(M)
            class_loss=bce_loss(C_logits,y)
            
            total_G_loss = adversarial_G_loss + opt.alpha*reconstruct_loss+class_loss*opt.beta

            class_loss_sum += class_loss.item()
            reconstruct_loss_sum += reconstruct_loss.item()
            adversarial_G_loss_sum += adversarial_G_loss.item()    

            total_G_loss.backward()

        writer.add_scalar("train/G_Loss/adversarial_G_loss", adversarial_G_loss_sum/accumulation_step, global_step)
        writer.add_scalar("train/G_Loss/reconstruct_loss", reconstruct_loss_sum/accumulation_step, global_step)
        writer.add_scalar("train/G_Loss/class_loss", class_loss_sum/accumulation_step, global_step)
        
        
        optimG.step() 
        optimC.step() 

        # -----------------
        #  Train  D
        # -----------------
        optimD.zero_grad()
        
        # update D only
        D_logit_imputed= netD(Hat_New_X.detach(), H,C_logits.detach() )
        D_loss= discriminator_loss(D_logit_imputed, M)
        train_loss_D +=D_loss.item()
        writer.add_scalar("train/D_Loss", D_loss,  global_step)
       
        D_loss.backward() 
        optimD.step() 
          
     
    
    netD.eval()
    netG.eval()
    netC.eval()

    all_logits  = []
    label_pred = []
    with torch.no_grad():
        for step_num, batch_data in enumerate(dev_dataloader):
            dev_X, dev_M, dev_y = tuple(t.to(device) for t in batch_data)
            dev_Z= Variable(FloatTensor(np.random.uniform(0, 1, (dev_X.shape[0], dim))))
            dev_New_X = dev_X * dev_M + (1-dev_M) * dev_Z
            dev_B = sample_binary(dev_X.shape[0], dim, 1-opt.p_hint)
            
            dev_B = torch.tensor(dev_B,device="cuda").float()
            dev_H = dev_M *dev_B+0.5*(1-dev_B)
            
            dev_G_sample = netG(dev_New_X,dev_M)
            dev_Hat_New_X = dev_New_X*dev_M +dev_G_sample * (1-dev_M)
            dev_pred_output = netC(dev_Hat_New_X)

            predict_loss += bce_loss(dev_pred_output, dev_y)

            numpy_logits = dev_pred_output.cpu().detach().numpy()
            label_pred += list((numpy_logits[:, 0] > opt.thold).astype(int) )
            all_logits += list(numpy_logits[:, 0])
            test_loss_mse +=test_loss(dev_X,dev_M,dev_G_sample).item()
            

    predict_loss=predict_loss/(step_num+1) 

    
    accuracy = np.equal(devy_tensor[:, 0], np.asarray(label_pred)).sum().detach().numpy() /  devy_tensor[:, 0].size().numel()

    f1=f1_score(devy[:, 0],np.array(label_pred),zero_division=0)
    recall=recall_score(devy[:, 0],np.array(label_pred),zero_division=0) 
    precision=precision_score(devy[:, 0],np.array(label_pred),zero_division=0)
    auc = roc_auc_score(devy[:, 0], np.array(all_logits))
    
   
    writer.add_scalar("devp/predict_loss", predict_loss, epoch_num+1)
    writer.add_scalar("devp/auc", auc, epoch_num+1)
    writer.add_scalar("devp/f1", f1, epoch_num+1)


    if f1>f1_max:
        f1_max=f1
    if auc>auc_max:
        auc_max=auc

    writer.close()
print(auc_max)
print(f1_max)

/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/random_ops.py:285: SyntaxWarning: "is" with a literal. Did you mean "=="?
  minval_is_zero = minval is 0  # pylint: disable=literal-comparison
/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/random_ops.py:286: SyntaxWarning: "is" with a literal. Did you mean "=="?
  maxval_is_one = maxval is 1  # pylint: disable=literal-comparison
/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/ragged/ragged_batch_gather_with_default_op.py:84: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (default_value.shape.ndims is not 0
/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/ragged/ragged_batch_gather_with_default_op.py:85: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and default_value.shape.ndims is not 1):


0.6630665229304227
0.692107545533391


# impute example

In [25]:
def impute(X,M):
    Z= Variable(FloatTensor(np.random.uniform(0, 1, (X.shape[0], dim))))
    New_X = M * X + (1-M) * Z  # Missing Data Introduce
    G_sample = netG(New_X,M)
    Hat_New_X=M*X +(1-M)*G_sample
    
    return Hat_New_X

In [26]:
testX = torch.tensor(testX,device=device).float()
testy= torch.tensor(testy,device=device).float()
#devZ_tensor = torch.tensor(devZ).float()
testM = torch.tensor(testX_M,device=device).float()

In [27]:
trgain_X=impute(trainX_tensor.to(device),trainM_tensor.to(device)).cpu().detach().numpy()